In [1]:
%matplotlib inline
# %matplotlib notebook
%config Completer.use_jedi = False
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


from scipy.special import logsumexp
from scipy.special import softmax
from scipy.special import betaln
from scipy.special import beta

from scipy.stats import multivariate_normal 
from scipy.stats import invwishart 

from sklearn import metrics
from scipy.special import psi
from scipy.special import digamma 


In [2]:
random_seed = 123
rng = np.random.default_rng(random_seed)

# Derivation

## Mean Field 

Mean Field (Structured) Independence Simplifying Assumption
$$\begin{align}
    q(\boldsymbol{\pi},\boldsymbol{\mu},\mathbf{z},\mathbf{X}_H)& = 
        \underbrace{q(\boldsymbol{\pi})}_{\text{Dirichlet}}\quad
        \underbrace{q(\boldsymbol{\mu})}_{\text{Beta}}\quad
        \underbrace{q(\mathbf{z})}_{\text{Cat}}\quad
        \underbrace{q(\mathbf{X}_H)}_{\text{Bernoulli}}\\

        &=  q(\mathbf{z}, \mathbf{X}_H) \cdot
            \prod_K^K q(\boldsymbol{\pi}_k) q(\boldsymbol{\mu}_k)
\end{align}$$

Update Steps 

$$\begin{align}
    &\ln q(\boldsymbol{\mu},\boldsymbol{\pi}) = \Big\langle \ln p(\mathbf{z},\mu,\mathbf{X}_H,\mathbf{X}_O) \Big\rangle_{q(\mathbf{z},\mathbf{X}_H)}  + \text{const}\\\\
    
    &\ln q(\mathbf{z},\mathbf{X}_H) = \Big\langle \ln p(\mathbf{z},\boldsymbol{\mu},\mathbf{X}_H,\mathbf{X}_O) \Big\rangle_{q(\boldsymbol{\mu},\boldsymbol{\pi})} + \text{const}
\end{align}$$

Posterior
$$
    p(\mathbf{X}_O,\mathbf{X}_H,\boldsymbol{\mu},\mathbf{z},\boldsymbol{\pi}) \propto p(\mathbf{X}_O,\mathbf{X}_H|\boldsymbol{\mu},\mathbf{z})p(\boldsymbol{\mu})p(\mathbf{z}|\boldsymbol{\pi})p(\boldsymbol{\pi})
$$

## Update for model parameters $p(\mu,\pi)$

$$\begin{align}
    \ln q(\boldsymbol{\mu},\boldsymbol{\pi}) &= \Big\langle \ln p(\mathbf{z},\boldsymbol{\mu},\mathbf{X}_H,\mathbf{X}_O) \Big\rangle_{q(\mathbf{z})} + \text{const}\\\\

    &= \Big\langle \ln p(\mathbf{X}_O,\mathbf{X}_H|\boldsymbol{\mu}, \mathbf{z}) \Big\rangle_{q(\mathbf{z})} + 
       \Big\langle \ln p(\boldsymbol{\mu}) \Big\rangle_{q(\mathbf{z})} + 
        \Big\langle \ln p(\mathbf{z}|\boldsymbol{\pi})p(\boldsymbol{\pi}) \Big\rangle_{q(\mathbf{z})} +
       \text{const}\\\\

    &=  \underbrace{\Big\langle \ln p(\mathbf{X}_O,\mathbf{X}_H|\boldsymbol{\mu}, \mathbf{z}) \Big\rangle_{q(\mathbf{z})} + 
        \ln p(\boldsymbol{\mu})}_{\text{Terms containing $\mu$}} + 
       \underbrace{
            \Big\langle \ln p(\mathbf{z}|\boldsymbol{\pi}) \Big\rangle_{q(\mathbf{z})} + 
            \ln p(\boldsymbol{\pi}) 
       }_{\text{\text{Terms containing $\boldsymbol{\pi}$}}} +
       \text{const}\\\\
\end{align}$$

#### Update Step for $q(\boldsymbol{\pi})$

$$\begin{align}
    \ln q(\boldsymbol{\pi}) &= \ln p(\boldsymbol{\pi}) + \Big\langle \ln p(\mathbf{z}|\boldsymbol{\pi}) \Big\rangle_{q(\mathbf{z})} + \text{const} \\

    &= \ln \text{Dir}(\boldsymbol{\pi}|\boldsymbol{\alpha}_0) + \Big\langle \sum_n^N \ln p(z_n|\boldsymbol{\pi}) \Big\rangle_{q(\mathbf{z})} + \text{const} \\

    &= \ln \Big[ \frac{1}{B(\alpha_0)}\prod_k^K \pi_k^{\alpha_0 - 1} \Big] + \Big\langle \sum_n^N \sum_k^K \ln \pi_{k}^{z_{nk}} \Big\rangle_{q(\mathbf{z})} + \text{const} \\
       
    &= \underbrace{-\ln B(\alpha_0)}_{\text{Independent of $\pi$}} + \sum_k^K \ln \pi_k^{\alpha_0 - 1} +
       \Big\langle \sum_k^K \sum_n^N z_{nk}\ln \pi_{k} \Big\rangle_{q(\mathbf{z})} + \text{const} \\

    &=   \sum_k^K (\alpha_0 - 1) \ln \pi_k + \sum_k^K \sum_n^N \Big\langle z_{nk} \Big\rangle_{q(\mathbf{z})} \ln \pi_{k} + \text{const}, \quad -\ln B(\alpha_0) \text{ absorbed into const}\\

    &= \sum_k^K (\alpha_0 - 1) \ln \pi_k + \sum_k^K \sum_n^N r_{nk} \ln \pi_{k} + \text{const}\\

    &= \sum_k^K \Big[ (\alpha_0 - 1) \ln \pi_k + \ln \pi_{k} \sum_n^N r_{nk} \Big] + \text{const}\\

    &= \sum_k^K \Big[ (\alpha_0 + N_k -1) \ln \pi_k  \Big] + \text{const}, \quad \text{where } N_k = \sum_n^N r_{nk} \\

    \implies & \boxed{q(\boldsymbol{\pi}) = \prod_k^K \pi_k^{\alpha_0 + N_k - 1} + \text{const} = \text{Dir}(\boldsymbol{\pi}|\boldsymbol{\alpha}_0 + N), \quad \text{where } N = [N_0,...,N_K]}\\
\end{align}$$

#### Update step for $q(\boldsymbol{\mu})$

$$\begin{align}
    \ln q(\boldsymbol{\mu}) &= \Big\langle \ln p(\mathbf{X}_H,\mathbf{X}_O|\boldsymbol{\mu}, \mathbf{z}) \Big\rangle_{q(\mathbf{z},\mathbf{X}_H)} + \ln p(\boldsymbol{\mu}) + \text{const}\\\\

    &= \sum_k^K \ln p(\boldsymbol{\mu}_k) + \Big\langle \sum_n^N \sum_k^K \ln p(\mathbf{x}^n_{O},\mathbf{x}^n_{H}|\boldsymbol{\mu}_k)^{z_{nk}} \Big\rangle_{q(\mathbf{z},\mathbf{X}_H)}  + \text{const}\\\\

    &= \sum_k^K \ln p(\boldsymbol{\mu}_k) + \Big\langle \sum_n^N \sum_k^K z_{nk} \cdot \ln p(\mathbf{x}^n_{O},\mathbf{x}^n_{H}|\boldsymbol{\mu}_k) \Big\rangle_{q(\mathbf{z},\mathbf{X}_H)}  + \text{const}\\\\

    &= \sum_k^K \Bigg[ \ln p(\boldsymbol{\mu}_k) + \sum_n^N \Big\langle z_{nk} \cdot \ln p(\mathbf{x}^n_{O},\mathbf{x}^n_{H}|\boldsymbol{\mu}_k) \Big\rangle_{q(\mathbf{z},\mathbf{X}_H)}   \Bigg]+ \text{const}\\\\ 

    \implies & \ln q(\boldsymbol{\mu}) = \sum_k^K \ln q(\boldsymbol{\mu}_k)
\end{align}$$

Hence,
$$\begin{align}
    \ln q(\boldsymbol{\mu}_k) &= \ln p(\boldsymbol{\mu}_k) + \sum_n^N \Big\langle z_{nk} \cdot \ln p(\mathbf{x}^n_{O},\mathbf{x}^n_{H}|\boldsymbol{\mu}_k) \Big\rangle_{q(z_n,\mathbf{x}^n_H)} + \text{const}\\\\

    &= \ln p(\boldsymbol{\mu}_k) + \sum_n^N \Big\langle z_{nk} \Big\rangle_{q_(z_n)} \cdot  \Big\langle \ln p(\mathbf{x}^n_{O},\mathbf{x}^n_{H}|\boldsymbol{\mu}_k) \Big\rangle_{q(\mathbf{x}^n_H | z_n)} + \text{const}\\\\

    &= \ln p(\boldsymbol{\mu}_k) +  \sum_n^N r_{nk} \cdot   \Big\langle \ln p(\mathbf{x}^n_{O},\mathbf{x}^n_{H}|\boldsymbol{\mu}_k) \Big\rangle_{q(\mathbf{x}^n_H | z_n)} + \text{const}\\\\

    &= \ln \mathcal{Beta}(\boldsymbol{\mu}_k | \mathbf{a}_{0},\mathbf{b}_{0}) + 
        \sum_n^N r_{nk} \cdot \Big\langle \ln \mathcal{Bern}(\mathbf{x}^n_{O},\mathbf{x}^n_{H}|\boldsymbol{\mu}_k) \Big\rangle_{q(\mathbf{x}^n_H | z_n)} + \text{const}\\\\

    &= \sum_d^D \ln\mathcal{Beta}(\mu_{k,d}|a_{k,d}, b_{k,d}) + 
         \sum_n^N \sum_d^D r_{nk} \cdot \Big\langle \ln \mathcal{Bern}(\mathbf{x}^{n,d}_{O},\mathbf{x}^{n,d}_{H}|\mu_{k,d}) \Big\rangle_{q(\mathbf{x}^n_H | z_n)} + \text{const}\\\\

    \implies \ln q(\mu_{k,d}) &= \ln \mathcal{Beta}(\mu_{k,d}|\mathbf{a}_{0,d},\mathbf{b}_{0,d}) + \sum_n^N r_{nk}  \cdot \Big\langle \ln
        \mathcal{Bern}(x^{n,d}_O,x^{n,d}_H|\boldsymbol{\mu}_{k,d}) \Big\rangle_{q(\mathbf{x}^n_H | z_n)} + \text{const}\\\\

    &= (a_{0,d} - 1)\ln\mu_{k,d} + (b_{0,d} - 1)\ln(1 - \mu_{k,d}) + 
        \sum_n^N r_{nk} \cdot \Big\langle [x^{n,d}_O, x^{n,d}_H] \ln\mu_{k,d} + (1 - [x^{n,d}_O, x^{n,d}_H])\ln(1 - \mu_{k,d}) \Big\rangle_{q(\mathbf{x}^n_H | z_n)} + \text{const}\\\\

    &= (a_{0,d} - 1)\ln \mu_{k,d} + (b_{0,d} - 1)\ln(1 - \mu_{k,d}) + 
        \sum_n^N r_{nk} \cdot \Big([x^{n,d}_O, \langle x^{n,d}_H\rangle ] \ln\mu_{k,d} + (1 - [x^{n,d}_O, \langle x^{n,d}_H \rangle ])\ln(1 - \mu_{k,d})\Big) + \text{const}\\\\
\end{align}$$

Where for $d \in H$ 
$$
    \langle x_{n,d} \rangle_{q(\mathbf{x}^n_H | z_n)} = \frac{e^{\langle \ln \mu_{k,d}\rangle}}{e^{\langle \ln \mu_{k,d} \rangle} + e^{\langle \ln(1 - \mu_{k,d}) \rangle} } =
    \frac{e^{\psi(a_{k,d}) - \psi(a_{k,d} + b_{k,d})}}{e^{\psi(a_{k,d}) - \psi(a_{k,d} + b_{k,d})} + e^{\psi(b_{k,d}) - \psi(a_{k,d} + b_{k,d})}}
$$

#### Update step for $q(\mathbf{z},\mathbf{X}_H)$
Noting that 
$$
    q(\mathbf{z},\mathbf{X}_H) = \underbrace{q(\mathbf{z})}_{(1)}\underbrace{q(\mathbf{X}_H|\mathbf{z})}_{(2)}
$$

We have
$$\begin{align}
    (2) \quad \ln q(\mathbf{X}_H|\mathbf{z}) &= \ln q(\mathbf{X}_H,\mathbf{z}) + \text{const}\\\\

    &= \Big\langle \ln p(\mathbf{X}_O,\mathbf{X}_H,\boldsymbol{\mu},\boldsymbol{\pi}) \Big\rangle_{q(\mu,\pi)} + \text{const}\\\\

     &= \Big\langle \ln p(\mathbf{X}_O,\mathbf{X}_H|\boldsymbol{\mu},\mathbf{z}) \Big\rangle_{q(\mu)} + 
       \underbrace{ \Big\langle \ln p(\mathbf{z}|\boldsymbol{\pi}) \Big\rangle_{q(\pi)} + 
        \ln p(\boldsymbol{\mu}) + \ln p(\boldsymbol{\pi})}_{\text{Independent of }\mathbf{X}_H} + 
    \text{const}\\\\

    &= \Big\langle \ln p(\mathbf{X}_O,\mathbf{X}_H|\boldsymbol{\mu},\mathbf{z}) \Big\rangle_{q(\mu)} + 
        \text{const}\\\\

    &= \Big\langle \sum_n^N \sum_k^K z_{nk} \cdot \ln p(\mathbf{x}^n_O,\mathbf{x}^n_H|\boldsymbol{\mu}_k) \Big\rangle_{q(\mu)} + \text{const}\\\\

    &= \sum_n^N  \Bigg\langle \sum_k^K z_{nk} \ln p(\mathbf{x}^n_O|\boldsymbol{\mu}_k) + z_{nk}\ln p(\mathbf{x}^n_{H}|\mathbf{x}^n_{O},\boldsymbol{\mu}_k)\Bigg\rangle_{q(\mu)} + \text{const}\\\\

    \implies & \ln q(\mathbf{x}^n_H|z_n) = \Bigg\langle \sum_k^K z_{nk} \ln p(\mathbf{x}^n_O|\boldsymbol{\mu}_k) + z_{nk}\ln p(\mathbf{x}^n_{H}|\mathbf{x}^n_{O},\boldsymbol{\mu}_k)\Bigg\rangle_{q(\mu)} + \text{const}\\\\

    &= \underbrace{\Bigg\langle \sum_k^K z_{nk} \ln p(\mathbf{x}^n_O|\boldsymbol{\mu}_k)\Bigg\rangle_{q(\mu)}}_{\text{Independent of }\mathbf{x}^n_H} + 
        \Bigg\langle \sum_k^K z_{nk}\ln p(\mathbf{x}^n_{H}|\mathbf{x}^n_{O},\boldsymbol{\mu}_k)\Bigg\rangle_{q(\mu)} + \text{const}\\\\

        &= \sum_k^K \Bigg\langle  z_{nk}\ln p(\mathbf{x}^n_{H}|\mathbf{x}^n_{O},\boldsymbol{\mu}_k)\Bigg\rangle_{q(\mu)} + \text{const}\\\\

    \implies \ln q(\mathbf{x}^n_H|z_n=k) &=  \Bigg\langle \ln p(\mathbf{x}^n_{H}|\mathbf{x}^n_{O},\boldsymbol{\mu}_k)\Bigg\rangle_{q(\mu)} + \text{const}\\\\

    &= \Bigg\langle \ln\mathcal{Bern}(\mathbf{x}^n_{H}|\mathbf{x}^n_{O},\boldsymbol{\mu}_k)\Bigg\rangle_{q(\mu)} + \text{const} =  \Bigg\langle \ln \mathcal{Bern}(\mathbf{x}^n_{H}|\boldsymbol{\mu}_k)\Bigg\rangle_{q(\mu)} + \text{const}
        ,\qquad \text{Since for Bernouilli } \mathbf{x}_H \perp \mathbf{x}_O \\\\

    &= \Bigg\langle \sum_{d\in H} x_d^n \ln \mu_{k,d} + (1 - x^n_d)\ln(1 - \mu_{k,d}) \Bigg\rangle_{q(\mu)} + \text{const}\\\\

    &= \sum_{d\in H} \Bigg[ x_d^n \Big\langle \ln \mu_{k,d} \Big\rangle_{q(\mu)} + 
        (1 - x^n_d) \Big\langle \ln (1-\mu_{k,d})\Big\rangle_{q(\mu)} \Bigg] +
        \text{const}\\\\

    \implies  \ln q(x_d^n | z_n =k) &= x_d^n \Big\langle \ln \mu_{k,d} \Big\rangle_{q(\mu)} + 
        (1 - x^n_d) \Big\langle \ln (1-\mu_{k,d})\Big\rangle_{q(\mu)}  + \text{const}\\\\

    &= x_d^n \Big( \psi(a_{k,d}) - \psi(a_{k,d} + b_{k,d}) \Big) + (1 - x_d^n)\Big(  \psi(b_{k,d}) - \psi(a_{k,d} + b_{k,d}) \Big)

\end{align}$$

  
And  
$$\begin{align}
    (1) \quad \ln q(z_n) &= \frac{q(z_n, \mathbf{x}_H^n)}{q(\mathbf{x}_H^n|z_n)}\\\\

    &= \ln q(z_n, \mathbf{x}^n_H) - \ln q(\mathbf{x}_H^n) + \text{const}\\\\

    &= \Big\langle \sum_k^K z_{nk} \ln \boldsymbol{\pi}_k + \sum_k^K z_{nk} \ln p(\mathbf{x}_O^n,\mathbf{x}_H^n|\boldsymbol{\mu}_k)  \Big\rangle_{q(\mu,\pi)} - 
        \Big\langle \sum_k^K z_{nk} \ln p(\mathbf{x}^n_H|\mathbf{x}^n_O,\boldsymbol{\mu}_k) \Big\rangle_{q(\mu)} + \text{const}\\\\

    &= \Big\langle  
            \sum_k^K z_{nk}\ln \boldsymbol{\pi}_k + 
            \sum_k^K z_{nk} \ln p(\mathbf{x}^n_O|\boldsymbol{\mu}_k) + 
            \sum_k^K z_{nk} \ln p(\mathbf{x}^n_H|\mathbf{x}^n_O,\boldsymbol{\mu}_k) 
       \Big\rangle_{q(\mu,\pi)} -
       \Big\langle 
            \sum_k^K z_{nk} \ln p(\mathbf{x}^n_H|\mathbf{x}^n_O,\boldsymbol{\mu}_k)
       \Big\rangle_{q(\mu)} + \text{const}\\\\

   
\end{align}$$

# Implementation

In [ ]:
def update_Θ(X,R)